# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770324799816                   -0.53    9.0         
  2   -2.771682536670       -2.87       -1.30    1.0    134ms
  3   -2.771714132461       -4.50       -2.63    1.0    112ms
  4   -2.771714713704       -6.24       -4.06    2.0    143ms
  5   -2.771714715218       -8.82       -4.76    2.0    136ms
  6   -2.771714715248      -10.52       -5.74    1.0   97.1ms
  7   -2.771714715250      -11.82       -6.29    2.0    128ms
  8   -2.771714715250      -13.89       -7.42    1.0    101ms
  9   -2.771714715250      -14.75       -7.62    2.0    124ms
 10   -2.771714715250   +  -14.35       -8.05    1.0    122ms


-0.00013457377961038485

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770470676940                   -0.52    9.0         
  2   -2.771777883610       -2.88       -1.33    1.0    119ms
  3   -2.771801530201       -4.63       -2.42    1.0   93.3ms
  4   -2.771801990974       -6.34       -3.10    1.0    120ms
  5   -2.771802074299       -7.08       -4.46    2.0    156ms
  6   -2.771802074430       -9.88       -4.70    1.0   99.2ms
  7   -2.771802074472      -10.37       -5.34    2.0    113ms
  8   -2.771802074475      -11.50       -5.80    1.0    125ms
  9   -2.771802074476      -12.21       -6.51    2.0    119ms
 10   -2.771802074476      -14.03       -6.82    1.0    119ms
 11   -2.771802074476      -14.18       -8.43    1.0    113ms


0.0176122188987487

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457377961038485
Displaced dipole:  0.0176122188987487
Polarizability :   1.7746792678359085


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920760398e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551378288e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852766285049e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694603318968e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088787880168e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.286348992312e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 1.081622794614e-12
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 1.081622794614e-12
[ Info: GMRES linsolve in iter 2; step 1: normres = 9.895479912383e-11
[ Info: GMRES linsolve in iter 2; step 2: normres = 1.480081271280e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 3.233814360249e-13
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 3.233814360249e-13
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 3.233728844677785e-13
└ *  number of operat

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.